<a href="https://colab.research.google.com/github/rajy4683/MonoMaskDepth/blob/master/DataGeneratorPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun May 10 16:36:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pwd

/content


In [0]:
!folder_list=("classroom" "college_outdoors" "corridors" "dining_room" "malls" "lobby" "meeting_rooms" "people_cropresize"); for folder_name in ${folder_list[@]}; do mkdir -p /content/S15EVA4/$folder_name /content/S15EVA4/$folder_name/aug_images; cp -Rf /content/drive/My\ Drive/EVA4/tsai/S15EVA4/$folder_name/*png /content/S15EVA4/$folder_name; done

### Dependencies


In [3]:
!pip install -r /content/drive/My\ Drive/EVA4/RekogNizer/requirements.txt

     |████████████████████████████████| 112kB 46.1MB/s 
     |████████████████████████████████| 1.4MB 44.9MB/s 
     |████████████████████████████████| 112kB 51.9MB/s 
     |████████████████████████████████| 102kB 15.0MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 460kB 53.3MB/s 
     |████████████████████████████████| 634kB 53.2MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=a9decda73f40e2de1b29642aaaa3a564f8208d1386b50fdd39fa6154563c141f
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=eb7e7b1c21e653537d76450aa7663813d89f335f5d9bc6660318c2cce6ecde90
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f

In [4]:
!pip install -U git+https://github.com/albu/albumentations

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-i8x4tm2p
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-i8x4tm2p
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=65055 sha256=98440d8183354ea0039a55d30b9280293eaecce15d1fea7441eb93e7a8ca4faa
  Stored in directory: /tmp/pip-ephem-wheel-cache-l63gemot/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [0]:
import os

import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.color import label2rgb

import albumentations as A
import random
from google.colab.patches import cv2_imshow
from PIL import Image
import glob
from zipfile import ZipFile


In [0]:
def load_return_fgbg_lists():
    base_dir = "/content/S15EVA4/"
    file_format = "*png"
    folder_list = ["classroom", "college_outdoors", 
                   "corridors", "dining_room",
                   "lobby","malls", "meeting_rooms",
                   "people_cropresize"]
    folder_dict = dict()
    for folder_name in folder_list:
        full_path = base_dir+folder_name+"/"+"*png"
        print(full_path)
        folder_dict[folder_name] = [ val for val in glob.glob(full_path)]
    return folder_dict


In [0]:
fg_bg_file_list = load_return_fgbg_lists()

/content/S15EVA4/classroom/*png
/content/S15EVA4/college_outdoors/*png
/content/S15EVA4/corridors/*png
/content/S15EVA4/dining_room/*png
/content/S15EVA4/lobby/*png
/content/S15EVA4/malls/*png
/content/S15EVA4/meeting_rooms/*png
/content/S15EVA4/people_cropresize/*png


In [0]:

BOX_COLOR = (255, 0, 0)
TEXT_COLOR = (255, 255, 255)

def visualize_bbox(img, bbox, color=BOX_COLOR, thickness=2, **kwargs):
    #height, width = img.shape[:2]

    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    return img

def visualize_titles(img, bbox, title, color=BOX_COLOR, thickness=2, font_thickness = 2, font_scale=0.35, **kwargs):
    #height, width = img.shape[:2]
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    
    ((text_width, text_height), _) = cv2.getTextSize(title, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(img, title, (x_min, y_min - int(0.3 * text_height)), cv2.FONT_HERSHEY_SIMPLEX, font_scale, TEXT_COLOR,
                font_thickness, lineType=cv2.LINE_AA)
    return img


def augment_and_show(aug, image, mask=None, bboxes=[], categories=[], category_id_to_name=[], filename=None, 
                     font_scale_orig=0.35, 
                     font_scale_aug=0.35, show_title=True, **kwargs):

    augmented = aug(image=image, mask=mask, bboxes=bboxes, category_id=categories)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_aug = cv2.cvtColor(augmented['image'], cv2.COLOR_BGR2RGB)

    for bbox in bboxes:
        visualize_bbox(image, bbox, **kwargs)

    for bbox in augmented['bboxes']:
        visualize_bbox(image_aug, bbox, **kwargs)

    if show_title:
        for bbox,cat_id in zip(bboxes, categories):
            visualize_titles(image, bbox, category_id_to_name[cat_id], font_scale=font_scale_orig, **kwargs)
        for bbox,cat_id in zip(augmented['bboxes'], augmented['category_id']):
            visualize_titles(image_aug, bbox, category_id_to_name[cat_id], font_scale=font_scale_aug, **kwargs)

    
    if mask is None:
        f, ax = plt.subplots(1, 2, figsize=(16, 8))
        
        ax[0].imshow(image)
        ax[0].set_title('Original image')
        
        ax[1].imshow(image_aug)
        ax[1].set_title('Augmented image')
    else:
        f, ax = plt.subplots(2, 2, figsize=(16, 16))
        
        if len(mask.shape) != 3:
            mask = label2rgb(mask, bg_label=0)            
            mask_aug = label2rgb(augmented['mask'], bg_label=0)
        else:
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            mask_aug = cv2.cvtColor(augmented['mask'], cv2.COLOR_BGR2RGB)
            
        ax[0, 0].imshow(image)
        ax[0, 0].set_title('Original image')
        
        ax[0, 1].imshow(image_aug)
        ax[0, 1].set_title('Augmented image')
        
        ax[1, 0].imshow(mask, interpolation='nearest')
        ax[1, 0].set_title('Original mask')

        ax[1, 1].imshow(mask_aug, interpolation='nearest')
        ax[1, 1].set_title('Augmented mask')

    f.tight_layout()
    if filename is not None:
        f.savefig(filename)
        
    return augmented['image'], augmented['mask'], augmented['bboxes']

def find_in_dir(dirname):
    return [os.path.join(dirname, fname) for fname in sorted(os.listdir(dirname))]

def generate_mask(img,debug=False):    
    lower_white = np.array([1, 1,1,4])
    upper_white = np.array([255,255,255,4])
    mask = cv2.inRange(img, lower_white, upper_white)
    if debug == True:
        cv2_imshow(img)
        cv2_imshow(mask)
    return mask
    



In [0]:
def image_overlay(background, foreground, x_offset, y_offset):
    #x_offset=y_offset=100

    #x1, x2 = x_offset, x_offset + foreground_img.shape[0]
    #y1, y2 = y_offset, y_offset + foreground_img.shape[1]

    background_full = Image.open(background)#.resize((250,250))
    foreground = Image.open(foreground).convert('RGBA')
    background_full.paste(foreground, (x_offset, y_offset), foreground)
    return background_full




In [0]:
random.seed(42)

#cv2.imread('images/parrot.jpg')

flip = A.Compose([
    A.IAAFliplr(p=1,always_apply=True),
    # A.RandomBrightnessContrast(p=1),    
    # A.RandomGamma(p=1),    
    # A.CLAHE(p=1),    
], p=1)

resize_bg = A.Compose([
    A.Resize(250,250,always_apply=True,p=1),
    #A.CLAHE(p=1),
    #A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50, p=1),
], p=1)


strong = A.Compose([
    A.ChannelShuffle(p=1),
], p=1)

In [0]:
def image_overlay_numpy(background, foreground, x_offset, y_offset):
    #x_offset=y_offset=100

    #x1, x2 = x_offset, x_offset + foreground_img.shape[0]
    #y1, y2 = y_offset, y_offset + foreground_img.shape[1]

    background_full = Image.fromarray(background)
    #Image.open(background)#.resize((250,250))
    foreground = Image.fromarray(foreground).convert('RGBA')
    #Image.open(foreground).convert('RGBA')
    background_full.paste(foreground, (x_offset, y_offset), foreground)
    return background_full.convert('RGB')

In [0]:
def rand_run_name():
    ran = random.randrange(10**80)
    myhex = "%064x" % ran
    #limit string to 64 characters
    myhex = myhex[:10]
    return myhex

In [0]:
def generate_mask(img,debug=False):    
    lower_white = np.array([1, 1,1])
    upper_white = np.array([255,255,255])
    mask = cv2.inRange(img[:,:,:-1], lower_white, upper_white)
    if debug == True:
        cv2_imshow(img)
        cv2_imshow(mask)
    return mask

def generate_fg_set(base_image_list, debug=False):
    for base_image_name in base_image_list:
        save_dir = os.path.join(os.path.dirname(base_image_name),"aug_images")
        basefile_name = os.path.basename(base_image_name)
        mask_name = "mask_"+basefile_name
        flip_name = "flip_"+basefile_name
        flip_mask_name = "mask_"+flip_name
        print(basefile_name, mask_name, flip_name,flip_mask_name)

        base_image = np.array(Image.open(base_image_name))
        base_mask = generate_mask(np.array(base_image))
        
        flipped_image = flip(image=base_image, mask=base_mask)
        Image.fromarray(base_mask).save(os.path.join(save_dir,mask_name))
        Image.fromarray(flipped_image['image']).save(os.path.join(save_dir,flip_name))
        Image.fromarray(flipped_image['mask']).save(os.path.join(save_dir,flip_mask_name))
        
        if (debug == True):
            display(Image.fromarray(base_image))
            display(Image.fromarray(base_mask))
            display(Image.fromarray(flipped_image['image']))
            display(Image.fromarray(flipped_image['mask']))




### Foreground with masks, flips and masks of flips

In [0]:
%cd /content/S15EVA4/

[Errno 2] No such file or directory: '/content/S15EVA4/'
/content


In [0]:
generate_fg_set(fg_bg_file_list['people_cropresize'])#[1]

In [0]:
def get_resize_aug(height_val, width_val):
    return A.Compose([
            A.Resize(height_val,width_val,always_apply=True,p=1),
            ], p=1)

In [0]:
def get_fg_image_masks(fg_file_name, bg_height, bg_width, debug=False):
    save_dir = os.path.join(os.path.dirname(fg_file_name),"aug_images")
    basefile_name = os.path.basename(fg_file_name)

    basefile = np.array(Image.open(fg_file_name))
    mask = np.array(Image.open(os.path.join(save_dir, "mask_"+basefile_name)))
    flip = np.array(Image.open(os.path.join(save_dir, "flip_"+basefile_name)))
    mask_flip = np.array(Image.open(os.path.join(save_dir, "mask_flip_"+basefile_name)))


    aspect_ratio = basefile.shape[0]/basefile.shape[1]
    final_fg_height = bg_height//2
    final_fg_width = np.int(final_fg_height // aspect_ratio)

    

    resized_orig_fg = get_resize_aug( final_fg_height, final_fg_width)(image=basefile, mask=mask)
    resized_flip_fg = get_resize_aug( final_fg_height, final_fg_width)(image=flip, mask=mask_flip)

    #print(resized_orig_fg['image'].shape)
    if (debug == True):
        print("Original shape:{} New Shape: {}".format(basefile.shape, (final_fg_height, final_fg_width)))
        display(Image.fromarray(basefile))
        display(Image.fromarray(resized_orig_fg['image']))
        display(Image.fromarray(resized_orig_fg['mask']))

        display(Image.fromarray(resized_flip_fg['image']))
        display(Image.fromarray(resized_flip_fg['mask']))

    return resized_orig_fg, resized_flip_fg

    #return os.path.join(save_dir, mask_name), os.path.join(save_dir, flip_name),  os.path.join(save_dir, mask_flip_name)



In [0]:
"""
  Given BG, FG and Flipped FG, retrieves/creates a new overlayed image and mask   
"""

def create_overlayed_img(background, 
                         foreground, 
                         flip_foreground, 
                         x_offset,
                         y_offset, 
                         bg_image, 
                         debug=False, 
                         save_images=True):

    save_dir = os.path.join(os.path.dirname(bg_image),"aug_images/")
    filename = os.path.basename(bg_image)
    counter = 1
    for val in [foreground, flip_foreground]:
        rand_str = str(rand_run_name())
        image = image_overlay_numpy(background['image'], val['image'], x_offset,y_offset)
        mask = image_overlay_numpy(background['mask'], val['mask'], x_offset,y_offset)
        if(debug == True):
            display(image)
            display(mask)
        if(save_images == True):
            image_save_path = os.path.join(save_dir+"image_"+rand_str+"_"+filename.replace("png","jpg"))
            mask_save_path = os.path.join(save_dir+"mask_"+rand_str+"_"+filename.replace("png","jpg"))
            #print(image_save_path,mask_save_path)
            image.save(image_save_path,'JPEG',quality=80)
            mask.save(mask_save_path,'JPEG',quality=80)


In [0]:
"""
    Top level-handler function:
    1. Calls albumentations to resize both BG, FG and Flipped FG
    2. Retrieves the resized masks for BG, FG and Flipped G
    3. Calls 'create_overlayed_img()' to blend/overlay the FG and Flipped FG on the BG 
"""
def overlay_images(background_path, foreground_path, 
                   bg_height=250,bg_width=250, 
                   replica_count=20, save_images=True,
                   debug=False):
    #print(background_path)
    #save_dir = os.path.join(os.path.dirname(fg_file_name),"aug_images")
    image = np.array(Image.open(background_path))
    # Masked output 
    mask_bg = np.zeros_like(image, dtype='uint8')
    
    resized_bg = get_resize_aug(bg_height,bg_width)(image=image, mask=mask_bg)    
    #print(resized_bg['image'].shape)
    
    
    resized_fg, resized_flip_fg  = get_fg_image_masks(foreground_path, bg_height, bg_width)
    ### Preventing FG to get thrown out of the FG
    max_y_offset = resized_bg['image'].shape[0]-resized_fg['image'].shape[0]
    max_x_offset = resized_bg['image'].shape[1]-resized_fg['image'].shape[1]
    randn_y = np.random.randint(0, max_y_offset, replica_count)
    randn_x = np.random.randint(0, max_x_offset, replica_count)
    for tuple_val in zip(randn_y,randn_x):
        y_offset=tuple_val[0]
        x_offset=tuple_val[1]
        create_overlayed_img(resized_bg, resized_fg, resized_flip_fg,
                             x_offset, y_offset, background_path, debug=debug, save_images=save_images)

In [0]:
fg_bg_file_list.keys()

dict_keys(['classroom', 'college_outdoors', 'corridors', 'dining_room', 'lobby', 'malls', 'meeting_rooms', 'people_cropresize'])

In [0]:
large_bg_file_list = []
for iter in list(fg_bg_file_list.keys())[:-1]:
    large_bg_file_list.extend(fg_bg_file_list[iter])

In [0]:
large_file_list = [ (bg, fg) for bg in large_bg_file_list for fg in fg_bg_file_list['people_cropresize']]

In [0]:
len(large_bg_file_list),len(large_file_list)

(0, 0)

### Triggers the mammoth overlay operations

In [0]:
from tqdm import tqdm
import os
import pandas as pd

df_file_list = pd.read_csv("/content/drive/My Drive/EVA4/tsai/S15EVA4/fg_bg.csv")
pbar = tqdm(df_file_list.loc[8000:,["Background","Foreground"]].values)
counter = 0
file_count = 0
for background_file, foreground_file in pbar:
    counter=counter+1
    if(counter%100 == 0 ):
        file_count = len([ val for folder_name in list(fg_bg_file_list.keys())[:-1] for val in glob.glob("/content/S15EVA4/"+folder_name+"/aug_images/*jpg")])
        #print("Total files:{}".format(file_count))
    status_desc = "Currently handling: {}:{} {} File Count:{}".format(counter,os.path.basename(background_file), os.path.basename(foreground_file), file_count)
    pbar.set_description(status_desc)    
    overlay_images(background_file, foreground_file,replica_count=20, save_images=True, debug=False)

Currently handling: 4100:meetin_room_9.png people_188.png File Count:327920: 100%|██████████| 4100/4100 [18:17<00:00,  3.74it/s]


In [0]:
file_list = [ val for folder_name in list(fg_bg_file_list.keys())[:-1] for val in glob.glob("/content/S15EVA4/"+folder_name+"/aug_images/*jpg")]
print("Total files to be zipped:{}".format(len(file_list)))
from zipfile import ZipFile 
import os 
with ZipFile('my_python_files_8k12k.zip','w') as zip: 
    # writing each file one by one 
    for file in file_list: 
        zip.write(file)

Total files to be zipped:328000


In [0]:
input_zip = ZipFile("/content/drive/My Drive/EVA4/tsai/S15EVA4/my_python_files_1k2k.zip")

In [0]:
input_zip.

In [0]:
import re
from io import BytesIO
image_list = []
for name in input_zip.namelist()[:100]:
    if re.search("image_",name):
        print(name)
        img = Image.open(BytesIO((input_zip.read(name))))
        image_list.append(img)
        #display(img)
        #print(file)

In [0]:
image_list

In [0]:
!mv /content/S15EVA4/my_python_files_8k12k.zip /content/drive/My\ Drive/EVA4/tsai/S15EVA4

In [0]:
# from IPython.display import display 
# from PIL import Image
# import numpy
# import cv2
# import numpy as np
# from google.colab.patches import cv2_imshow
# #from common import Sketcher
 
# image = cv2.imread(people_list[3])
# #display(img)

# image_mark = image.copy()
# hsv = cv2.cvtColor(image_mark, cv2.COLOR_BGR2HSV)
# #sketch = Sketcher('Image', [image_mark], lambda : ((255, 255, 255), 255))

# gray = cv2.cvtColor(image_mark, cv2.COLOR_BGR2GRAY)
# print(image_mark.shape, gray.shape)
# cv2_imshow( image_mark)
# cv2_imshow( hsv)


# lower_white = np.array([0,0,255])
# upper_white = np.array([255,255,255])

# #mask = cv2.inRange(image_mark, lower_white, upper_white)
# mask = cv2.inRange(hsv, lower_white, upper_white)
# mask_inv = cv2.bitwise_not(mask)
# cv2_imshow( mask)
# cv2_imshow( mask_inv)



# # Extract the dimensions of the original image
# rows, cols, channels = image.shape
# image = image[0:rows, 0:cols]

# # # Bitwise-OR mask and original image


# # # Bitwise-OR inverse mask and grayscale image
# # gray_portion = cv2.bitwise_or(gray, gray, mask = mask_inv)
# # gray_portion = np.stack((gray_portion,)*3, axis=-1)

# # # Combine the two images
# # output = colored_portion# + gray_portion

# #mask = np.ones(shape=image.shape[0:2], dtype="bool")
# colored_portion = cv2.bitwise_or(image, image, mask = mask)
# colored_portion = colored_portion[0:rows, 0:cols]
# cv2_imshow( mask)
# cv2_imshow( colored_portion)


In [0]:
# import cv2
# import numpy as np

# img = cv2.imread(people_list[1])
# #img = cv2.imread('/content/drive/My Drive/EVA4/tsai/S15EVA4/people/mypeople_10.jpg')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# cv2_imshow(img)
# #cv2.waitKey(0)
# #we readed the image

# lower_white = np.array([1, 1,1])
# upper_white = np.array([255,255,255])

# mask = cv2.inRange(img, lower_white, upper_white)
# cv2_imshow(mask)


# mask_inv = cv2.bitwise_not(mask)
# cv2_imshow(mask_inv)
# # #mask = np.ones(img.shape[:2],dtype="uint8")
# #plain_white=np.full(img.shape,255, dtype=np.uint8)
# cv2_imshow(255-img)


# # cv2_imshow(plain_white)
# # #print(plain_white.shape)
# # mask = np.full((img.shape[0], img.shape[1]), 255, dtype=np.uint8)
# # #print(img.shape, mask.shape)
# # fg = cv2.bitwise_xor(img, plain_white, mask=mask)
# # cv2_imshow(fg)

# # # get second masked value (background) mask must be inverted
# # # mask = cv2.bitwise_not(mask)
# # # background = np.full(img.shape, 128, dtype=np.uint8)
# # # cv2_imshow(background)
# # # bk = cv2.bitwise_or(background, background, mask=mask)

# # # # combine foreground+background
# # # final = cv2.bitwise_or(fg, bk)
# # cv2_imshow(final)

# #img


In [0]:
%cd /content/drive/My\ Drive/EVA4

/content/drive/My Drive/EVA4


In [0]:
!git clone https://github.com/ialhashim/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 235 (delta 0), reused 1 (delta 0), pack-reused 231
Receiving objects: 100% (235/235), 11.80 MiB | 14.20 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O /content/nyu.h5

--2020-05-03 10:46:05--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.65.4
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.65.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘/content/nyu.h5’

/content/nyu.h5     100%[===================>] 164.89M  11.6MB/s    in 16s     

2020-05-03 10:46:22 (10.4 MB/s) - ‘/content/nyu.h5’ saved [172897376/172897376]



In [0]:
import re
input_zip = ZipFile("/content/drive/My Drive/EVA4/tsai/S15EVA4/my_python_files_1k2k.zip")


In [0]:
from io import BytesIO

def load_images_local(image_files):
    loaded_images = []
    for file_name in image_files:
        resized_img = get_resize_aug(480,640)(image=np.asarray(file_name))['image']        
        x = np.clip(np.array(resized_img, dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

main_zip_file_list = input_zip.namelist()[:2023]
batch_start = 0
batch_step = 1000

local_step = batch_step
max_iter = np.int(np.ceil(len(main_zip_file_list)/batch_step))
last_iter_delta = len(main_zip_file_list)%batch_step


print("Total iter:{} last_batch: {}".format(max_iter, last_iter_delta))

while(max_iter > 0):    
    if(max_iter == 1):
        local_step=last_iter_delta
    print("{}:Start: {} Stop: {}".format(max_iter, batch_start,batch_start+local_step))
    batch_start=batch_start+local_step
    max_iter-=1
    image_list=[]
    for name in input_zip.namelist()[batch_start:batch_start+local_step]:
        if re.search("image_",name):
            #print(name)
            img = Image.open(BytesIO((input_zip.read(name))))
            image_list.append(img)
        #display(img)
    
    my_local_stack = load_images_local(image_list)
    print(len(image_list),len(my_local_stack))
    

# while (remain > 0):
#     #
#     #inputs = load_images_local(image_list)
#     #outputs = predict(model, inputs)
#     #print("Start: {} Stop: {}".format(batch_start,batch_start+local_step))
    
#     if (remain - (2*batch_step)  > 0):        
#         local_step=batch_step
#     else:
#         #batch_start=batch_start+remain
#         local_step=remain
#         remain=
#     batch_start=batch_start+local_step
#     print("Start: {} Stop: {} Remain: {}".format(batch_start,batch_start+local_step, remain))    
    

Total iter:3 last_batch: 23
3:Start: 0 Stop: 1000
484 484
2:Start: 1000 Stop: 2000
488 488
1:Start: 2000 Stop: 2023
8 8


In [0]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('/content/drive/My Drive/EVA4/tsai/S15EVA4/my_python_files_1k2k.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content')

In [0]:
import re
input_zip = ZipFile("/content/drive/My Drive/EVA4/tsai/S15EVA4/my_python_files_1k2k.zip")
for name in input_zip.namelist()[:200]:
    if re.search("image_",name):
        #print(name)
        input_zip.extract(name,path='/content')
        #img = Image.open(BytesIO((input_zip.read(name))))

In [0]:
%cd /content/drive/My\ Drive/EVA4/DenseDepth

/content/drive/My Drive/EVA4/DenseDepth


In [0]:
!rm -rf /content/test

In [0]:
!ls -lrth /content/drive/My\ Drive/EVA4/DenseDepth/nyu.h5

-rw------- 1 root root 165M Feb  4  2019 '/content/drive/My Drive/EVA4/DenseDepth/nyu.h5'


In [0]:
!cp /content/drive/My\ Drive/EVA4/tsai/S15EVA4/my_python_files_8k12k.zip /content

In [0]:
!python test.py --model "/content/nyu.h5" --input "/content/my_python_files_8k12k.zip" --chunk_size 300

In [0]:
file_list = [ folder_name for folder_name in glob.glob("/content/test/*jpg")]
print("Total files to be zipped:{}".format(len(file_list)))
from zipfile import ZipFile 
import os 
with ZipFile('/content/depth_8k12k.zip','w') as zip: 
    # writing each file one by one 
    for file in file_list: 
        zip.write(file,arcname=os.path.basename(file))

Total files to be zipped:163700


In [0]:
import re
import pandas as pd
input_zip = ZipFile("/content/drive/My Drive/EVA4/tsai/S15EVA4/my_python_files_8k12k.zip")
file_list = [(os.path.basename(name).replace(".jpg",""),name) for name in input_zip.namelist() if re.search("image_",name) ]
df_file_list = pd.DataFrame(file_list, columns=["BaseName","FullName"])

depth_zip = ZipFile("/content/depth_8k12k.zip")
depth_file_list = [ os.path.basename(name).replace("depth_","").replace(".jpg","") for name in depth_zip.namelist()]
missing_file_list =(df_file_list.loc[~df_file_list.BaseName.isin(depth_file_list),'FullName'].values)

In [0]:
sys.path.append('/content/drive/My Drive/EVA4/')
sys.path.append('/content/drive/My Drive/EVA4/RekogNizer')
sys.path.append('/content/drive/My Drive/EVA4/DenseDepth')

In [0]:
%load_ext autoreload
%autoreload 1
from DenseDepth import utils#load_execute_names
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model('/content/nyu.h5', custom_objects=custom_objects, compile=False)
utils.load_execute_names(model, input_zip, missing_file_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Using TensorFlow backend.


Loading model...
Loaded (300) images of size (480, 640, 3).
Received output shape:(300, 240, 320, 1) converted:(300, 240, 320, 3)


In [0]:
len(ZipFile("/content/depth_8k12k.zip").namelist()), len(ZipFile("/content/drive/My Drive/EVA4/tsai/S15EVA4/depth_8k12k.zip").namelist())


(164000, 163700)

In [0]:
file_list = [ folder_name for folder_name in glob.glob("/content/test/*jpg")]
print("Total files:{}".format(len(file_list)))
from zipfile import ZipFile 
import os 
with ZipFile('/content/depth_8k12k.zip','a') as zip: 
    # writing each file one by one 
    for file in file_list: 
        zip.write(file,arcname=os.path.basename(file))
zip.close()

Total files:300


In [0]:
!cp /content/depth_8k12k.zip /content/drive/My\ Drive/EVA4/tsai/S15EVA4

In [0]:
!cp 

In [0]:
!python test.py --model "/content/nyu.h5" --input "/content/my_python_files_2k6k.zip" --chunk_size 300

In [0]:
%matplotlib inline
!python test.py --model "/content/drive/My Drive/EVA4/DenseDepth/nyu.h5" 

Using TensorFlow backend.
Loading model...

Model loaded (/content/drive/My Drive/EVA4/DenseDepth/nyu.h5).

Loaded (12) images of size (480, 640, 3).
<Figure size 1000x500 with 1 Axes>


In [0]:
!rm -rf /content/content/

In [0]:
!ls -1 /content/content/S15EVA4/classroom/aug_images|wc -l

88


In [0]:
zf = ZipFile('/content/zipfile_append.zip', mode='w')
try:
    zf.write('/content/sample_data/README.md')
finally:
    zf.close()

In [0]:
for image_file in glob.glob('/content/content/S15EVA4/classroom/aug_images/*jpg'):
    print(image_file)
    savedir = os.path.dirname(image_file)
    new_file = os.path.join(savedir, "res_"+os.path.basename(image_file))
    img = np.array(Image.open(image_file))
    resized_img = get_resize_aug(480,640)(image=img)
    Image.fromarray(resized_img['image']).save(new_file)


In [0]:
import tensorflow as tf

In [0]:
tf.image.ResizeMethod.BILINEAR

'bilinear'

In [0]:
tf.image.resize(images, size, method='bilinear', preserve_aspect_ratio=False, antialias=False, name=None)

In [0]:
!rm -rf /content/test